In [1]:
import os
import datetime
from datetime import timedelta
import quandl
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import plotnine as p9
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore

import warnings
import functools
import wrds
import sys
# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
db = wrds.Connection()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
earnings_dates_for_tickers = pd.read_csv("earnings_dates_for_tickers.csv")

In [4]:
csv_file_path = "secids.csv"

# Reading the CSV file into a NumPy array
secids = np.genfromtxt(csv_file_path, delimiter=',')
secids

array([101149., 205456., 210354., ..., 211262., 207854., 105959.])

In [5]:
options_dict = {} 
for year in range(2017, 2024):
    print(year)
    table_name = f"optionm.opprcd{year}"  # Generate table name dynamically
    query = f"""
    SELECT
        date, secid, symbol, cp_flag, expiry_indicator, volume, strike_price, exdate, open_interest, impl_volatility, best_bid, best_offer, delta, gamma, vega, theta
    FROM 
        {table_name} a 
    WHERE
        a.secid IN ({','.join(map(str, secids))}) AND
        a.volume > 0 AND
        a.exdate - a.date <= 7
    """
    options_dict[year] = db.raw_sql(query, date_cols=['date'])

2017
2018
2019
2020
2021
2022
2023


In [6]:
for year in range(2017, 2024):
    # Get the DataFrame for the current year
    options = options_dict[year]
    
    # Split the 'symbol' column and extract the first substring as 'ticker'
    options['ticker'] = options['symbol'].str.split(' ').str[0]
    
    # Set the multi-index using 'date' and 'ticker'
    options.set_index(['date', 'ticker'], inplace=True)
    
    # Update the options_dict with the modified DataFrame
    options_dict[year] = options

In [7]:
stock_prices = pd.read_parquet("stock_price_ED_20182023.parquet")
stock_prices

,date,permno,prc,bid,ask,ticker,comnam,permco
0,2018-02-07,87432,68.06,68.06000,68.07000,A,AGILENT TECHNOLOGIES INC,36364.0
1,2018-02-08,87432,65.05,65.03000,65.05000,A,AGILENT TECHNOLOGIES INC,36364.0
2,2018-02-09,87432,66.98,66.97000,66.98000,A,AGILENT TECHNOLOGIES INC,36364.0
3,2018-02-12,87432,68.43,68.40000,68.41000,A,AGILENT TECHNOLOGIES INC,36364.0
4,2018-02-13,87432,68.34,68.33000,68.34000,A,AGILENT TECHNOLOGIES INC,36364.0
...,...,...,...,...,...,...,...,...
147109,2023-11-03,13788,162.23,162.19000,162.23000,ZTS,ZOETIS INC,54327.0
147110,2023-11-06,13788,163.17,163.20000,163.27000,ZTS,ZOETIS INC,54327.0
147111,2023-11-07,13788,167.16,167.24001,167.28000,ZTS,ZOETIS INC,54327.0
147112,2023-11-08,13788,170.87,170.89000,170.92999,ZTS,ZOETIS INC,54327.0


**Sectors**

In [34]:
sectors_filter = pd.read_excel('tickers_by_sector.xlsx', sheet_name='earnings_dates_for_tickers')
sectors = pd.read_excel('tickers_by_sector.xlsx', sheet_name='Sheet1')

In [35]:
sectors = sectors.drop(sectors.index[0])
sectors.columns = sectors.iloc[0]
sectors = sectors.drop(sectors.index[0])

In [36]:
sectors = sectors.rename(columns={'short ticker': 'ticker'})
sectors.columns

Index(['Ticker', 'Short Name', 'Market Cap', 'Price:D-1', 'P/E',
       'Total Return YTD', 'Revenue T12M', 'EPS T12M', 'ticker new format',
       'ticker', 'SECTOR'],
      dtype='object', name=1)

In [37]:
sectors_tickers = sectors[['ticker', 'SECTOR']]
sectors_tickers

1,ticker,SECTOR
2,NaN,NaN
3,GOOGL,Communication Services (83 securities)
4,META,Communication Services (83 securities)
5,NFLX,Communication Services (83 securities)
6,DIS,Communication Services (83 securities)
...,...,...
3011,HE,Utilities (63 securities)
3012,MSEX,Utilities (63 securities)
3013,UTL,Utilities (63 securities)
3014,NaN,NaN


In [38]:
stock_prices.columns

Index(['date', 'permno', 'prc', 'bid', 'ask', 'ticker', 'comnam', 'permco'], dtype='object')

In [39]:
merged_df = pd.merge(stock_prices, sectors_tickers, on='ticker', how='inner')
merged_df

,date,permno,prc,bid,ask,ticker,comnam,permco,SECTOR
0,2018-02-07,87432,68.06,68.06000,68.07000,A,AGILENT TECHNOLOGIES INC,36364.0,Health Care (302 securities)
1,2018-02-08,87432,65.05,65.03000,65.05000,A,AGILENT TECHNOLOGIES INC,36364.0,Health Care (302 securities)
2,2018-02-09,87432,66.98,66.97000,66.98000,A,AGILENT TECHNOLOGIES INC,36364.0,Health Care (302 securities)
3,2018-02-12,87432,68.43,68.40000,68.41000,A,AGILENT TECHNOLOGIES INC,36364.0,Health Care (302 securities)
4,2018-02-13,87432,68.34,68.33000,68.34000,A,AGILENT TECHNOLOGIES INC,36364.0,Health Care (302 securities)
...,...,...,...,...,...,...,...,...,...
142742,2023-11-03,13788,162.23,162.19000,162.23000,ZTS,ZOETIS INC,54327.0,Health Care (302 securities)
142743,2023-11-06,13788,163.17,163.20000,163.27000,ZTS,ZOETIS INC,54327.0,Health Care (302 securities)
142744,2023-11-07,13788,167.16,167.24001,167.28000,ZTS,ZOETIS INC,54327.0,Health Care (302 securities)
142745,2023-11-08,13788,170.87,170.89000,170.92999,ZTS,ZOETIS INC,54327.0,Health Care (302 securities)


In [8]:
earnings_dates_for_tickers['trade_entry'] = pd.to_datetime(earnings_dates_for_tickers['trade_entry'])

# Extract only the date part (YYYY-MM-DD)
earnings_dates_for_tickers['trade_entry'] = earnings_dates_for_tickers['trade_entry'].dt.date
earnings_dates_for_tickers['trade_entry']

0        2018-02-14
1        2018-05-14
2        2018-08-14
3        2018-11-19
4        2019-02-20
            ...    
19471    2023-02-13
19472    2023-05-03
19473    2023-08-07
19474    2023-11-01
19475    2024-02-12
Name: trade_entry, Length: 19476, dtype: object

In [9]:
earnings_dates_for_tickers['trade_entry'] = pd.to_datetime(earnings_dates_for_tickers['trade_entry'])
earnings_dates_for_tickers['trade_entry']


0       2018-02-14
1       2018-05-14
2       2018-08-14
3       2018-11-19
4       2019-02-20
           ...    
19471   2023-02-13
19472   2023-05-03
19473   2023-08-07
19474   2023-11-01
19475   2024-02-12
Name: trade_entry, Length: 19476, dtype: datetime64[ns]

In [10]:
earnings_dates_for_tickers['trade_exit'] = pd.to_datetime(earnings_dates_for_tickers['trade_exit'])

# Extract only the date part (YYYY-MM-DD)
earnings_dates_for_tickers['trade_exit'] = earnings_dates_for_tickers['trade_exit'].dt.date
earnings_dates_for_tickers['trade_exit'] = pd.to_datetime(earnings_dates_for_tickers['trade_exit'])
earnings_dates_for_tickers['trade_exit']



0       2018-02-15
1       2018-05-15
2       2018-08-15
3       2018-11-20
4       2019-02-21
           ...    
19471   2023-02-14
19472   2023-05-04
19473   2023-08-08
19474   2023-11-02
19475   2024-02-13
Name: trade_exit, Length: 19476, dtype: datetime64[ns]

In [11]:
stock_prices.set_index(['date','ticker'],inplace = True)
stock_prices

,,permno,prc,bid,ask,comnam,permco
date,ticker,,,,,,
2018-02-07,A,87432,68.06,68.06000,68.07000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-08,A,87432,65.05,65.03000,65.05000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-09,A,87432,66.98,66.97000,66.98000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-12,A,87432,68.43,68.40000,68.41000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-13,A,87432,68.34,68.33000,68.34000,AGILENT TECHNOLOGIES INC,36364.0
...,...,...,...,...,...,...,...
2023-11-03,ZTS,13788,162.23,162.19000,162.23000,ZOETIS INC,54327.0
2023-11-06,ZTS,13788,163.17,163.20000,163.27000,ZOETIS INC,54327.0
2023-11-07,ZTS,13788,167.16,167.24001,167.28000,ZOETIS INC,54327.0


In [12]:
def entry_stock_price(row):
    try:
        return stock_prices.loc[(row['trade_entry'], row['Short_ticker'])]['prc']
    except KeyError:
        return np.nan
earnings_dates_for_tickers['stock_price_entry'] = earnings_dates_for_tickers.apply(entry_stock_price,axis = 1)

In [13]:
def exit_stock_price(row):
    try:
        return stock_prices.loc[(row['trade_exit'], row['Short_ticker'])]['prc']
    except KeyError:
        return np.nan
earnings_dates_for_tickers['stock_price_exit'] = earnings_dates_for_tickers.apply(exit_stock_price,axis = 1)

In [14]:
earnings_dates_for_tickers['year_entry'] = earnings_dates_for_tickers['trade_entry'].dt.year

# Create 'year_exit' column containing the year of 'trade_exit'
earnings_dates_for_tickers['year_exit'] = earnings_dates_for_tickers['trade_exit'].dt.year
earnings_dates_for_tickers['year_entry']

0        2018
1        2018
2        2018
3        2018
4        2019
         ... 
19471    2023
19472    2023
19473    2023
19474    2023
19475    2024
Name: year_entry, Length: 19476, dtype: int32

In [15]:
def straddle_entry_mid(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan
    try:
        options = options_dict[row['year_entry']].loc[(row['trade_entry'],row['Short_ticker'])]
    except:
        return np.nan
    options['strike_price'] *= 0.001

    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['abs_difference'] = abs(options['strike_price'] - row['stock_price_entry'])

    # Filter the DataFrame for rows with the minimum absolute difference
    min_abs_difference = options['abs_difference'].min()
    filtered_options = options[options['abs_difference'] == min_abs_difference]
    if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
        return (filtered_options['best_bid'].mean()+filtered_options['best_offer'].mean())/2.0
    return np.nan
earnings_dates_for_tickers['straddle_entry_mid'] = earnings_dates_for_tickers.apply(straddle_entry_mid,axis = 1)

In [16]:
earnings_dates_for_tickers

,Unnamed: 0,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry,trade_exit,stock_price_entry,stock_price_exit,year_entry,year_exit,straddle_entry_mid
0,0,396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00,2018-02-14,2018-02-15,69.70,72.02000,2018,2018,1.1825
1,1,1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00,2018-05-14,2018-05-15,69.21,62.50000,2018,2018,1.6850
2,2,2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00,2018-08-14,2018-08-15,66.75,64.73000,2018,2018,2.2750
3,3,2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00,2018-11-19,2018-11-20,62.61,67.52000,2018,2018,NaN
4,4,3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00,2019-02-20,2019-02-21,78.56,77.89000,2019,2019,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19471,19471,15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,2023-02-13 00:00:00+00:00,2023-02-13,2023-02-14,163.14,171.89999,2023,2023,3.0750
19472,19472,16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,2023-05-03 00:00:00+00:00,2023-05-03,2023-05-04,177.73,178.47000,2023,2023,NaN
19473,19473,17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,2023-08-07 00:00:00+00:00,2023-08-07,2023-08-08,180.69,189.30000,2023,2023,NaN
19474,19474,18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,2023-11-01 00:00:00+00:00,2023-11-01,2023-11-02,151.44,160.91000,2023,2023,NaN


In [17]:
stop

NameError: name 'stop' is not defined

In [ ]:
def straddle_exit_mid(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan
    try:
        options = options_dict[row['year_exit']].loc[(row['trade_exit'],row['Short_ticker'])]
    except:
        return np.nan
    options['strike_price'] *= 0.001

    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['abs_difference'] = abs(options['strike_price'] - row['stock_price_entry'])

    # Filter the DataFrame for rows with the minimum absolute difference
    min_abs_difference = options['abs_difference'].min()
    filtered_options = options[options['abs_difference'] == min_abs_difference]
    if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
        return (filtered_options['best_bid'].mean()+filtered_options['best_offer'].mean())/2.0
    return np.nan
earnings_dates_for_tickers['straddle_exit_mid'] = earnings_dates_for_tickers.apply(straddle_exit_mid,axis = 1)

In [ ]:
earnings_dates_for_tickers['straddle_exit_mid'].count()

5288

In [ ]:
earnings_dates_for_tickers['straddle_pnl'] = (earnings_dates_for_tickers['straddle_entry_mid'] - earnings_dates_for_tickers['straddle_exit_mid'])*1000.0/earnings_dates_for_tickers['straddle_entry_mid']

In [ ]:
earnings_dates_for_tickers['straddle_pnl'].mean()

-26.64774755013799